# From Quality control to Clustering for day 5 samples

package loading

In [1]:
library(ggplot2)
library(cowplot)
library(dplyr)
library(tibble)
library(AnnotationHub)
library(patchwork)
library(Seurat)
library(harmony)
library(magrittr)
library(SeuratWrappers)
library(gprofiler2)


Attachement du package : ‘dplyr’


Les objets suivants sont masqués depuis ‘package:stats’:

    filter, lag


Les objets suivants sont masqués depuis ‘package:base’:

    intersect, setdiff, setequal, union


Le chargement a nécessité le package : BiocGenerics


Attachement du package : ‘BiocGenerics’


Les objets suivants sont masqués depuis ‘package:dplyr’:

    combine, intersect, setdiff, union


Les objets suivants sont masqués depuis ‘package:stats’:

    IQR, mad, sd, var, xtabs


Les objets suivants sont masqués depuis ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Le chargement a nécessité le package : BiocFile

Definitions of colors pallet

In [2]:
custom_colors <- list()

colors_dutch <- c(
  '#FFC312','#C4E538','#12CBC4','#FDA7DF','#ED4C67',
  '#F79F1F','#A3CB38','#1289A7','#D980FA','#B53471',
  '#EE5A24','#009432','#0652DD','#9980FA','#833471',
  '#EA2027','#006266','#1B1464','#5758BB','#6F1E51'
)

colors_spanish <- c(
  '#40407a','#706fd3','#f7f1e3','#34ace0','#33d9b2',
  '#2c2c54','#474787','#aaa69d','#227093','#218c74',
  '#ff5252','#ff793f','#d1ccc0','#ffb142','#ffda79',
  '#b33939','#cd6133','#84817a','#cc8e35','#ccae62',
  '#45aaf2', '#7f8c8d'
)

colors_turkish <- c(
  '#cd84f1','#ffcccc','#ff4d4d','#ffaf40','#fffa65',
  '#32ff7e','#7efff5','#18dcff','#7d5fff','#4b4b4b'
)

custom_colors$discrete <- c(colors_dutch, colors_spanish, colors_turkish)

custom_colors$cell_cycle <- setNames(
  c('#45aaf2', '#f1c40f', '#e74c3c', '#7f8c8d'),
  c('G1',      'S',       'G2M',     '-')
)

plot resolution

In [3]:
options(repr.plot.width = 18, repr.plot.height = 17, repr.plot.pointsize = 24)

In [4]:
library(future)

In [5]:
plan("multicore", workers = 4)
plan()

multicore:
- args: function (..., workers = 4, envir = parent.frame())
- tweaked: TRUE
- call: plan("multicore", workers = 4)

Loading of the cell ranger matrix

In [58]:
j5_05122021.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/J5-1_arcref/outs/filtered_feature_bc_matrix")
j5_03062021.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/J5-2_arcref/outs/filtered_feature_bc_matrix")
j7_3.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/J7-3_arcref/outs/filtered_feature_bc_matrix")
j7_31082017.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/J7-1_arcref/outs/filtered_feature_bc_matrix")
j7_15112017.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/J7-2_arcref/outs/filtered_feature_bc_matrix")
j7_4.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/J7-4_arcref/outs/filtered_feature_bc_matrix")
j9_1.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/J9-3_arcref/outs/filtered_feature_bc_matrix")
j9_4.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/J9-2_arcref/outs/filtered_feature_bc_matrix")
j9_3.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/J9-1_arcref/outs/filtered_feature_bc_matrix")
j9_3e.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/J9-4_arcref/outs/filtered_feature_bc_matrix")
j11.1.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/J11-1_arcref/outs/filtered_feature_bc_matrix")
j11_2.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/J11-2_arcref/outs/filtered_feature_bc_matrix")
SS_E11_5_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E11_5_A_arcref/outs/filtered_feature_bc_matrix")
SS_E11_5_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E11_5_B_arcref/outs/filtered_feature_bc_matrix")
SS_E12_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E12_A_arcref/outs/filtered_feature_bc_matrix")
SS_E12_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E12_B_arcref/outs/filtered_feature_bc_matrix")
SS_E12_5_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E12_5_A_arcref/outs/filtered_feature_bc_matrix")
SS_E12_5_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E12_5_B_arcref/outs/filtered_feature_bc_matrix")
SS_E13_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_A_arcref/outs/filtered_feature_bc_matrix")
SS_E13_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_B_arcref/outs/filtered_feature_bc_matrix")
SS_E13_C.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_C_arcref/outs/filtered_feature_bc_matrix")
SS_E13_D.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_D_arcref_2/outs/filtered_feature_bc_matrix")
SS_E13_E.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_E_arcref_2/outs/filtered_feature_bc_matrix")
SS_E13_5_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_5_A_arcref/outs/filtered_feature_bc_matrix")
SS_E13_5_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_5_B_arcref/outs/filtered_feature_bc_matrix")
SS_E13_5_C.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_5_C_arcref/outs/filtered_feature_bc_matrix")
SS_E13_5_D.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E13_5_D_arcref/outs/filtered_feature_bc_matrix")
SS_E14_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E14_A_arcref/outs/filtered_feature_bc_matrix")
SS_E14_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E14_B_arcref/outs/filtered_feature_bc_matrix")
SS_E14_5_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E14_5_B_arcref/outs/filtered_feature_bc_matrix")
SS_E14_5_C.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E14_5_C_arcref/outs/filtered_feature_bc_matrix")
SS_E14_5_D.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E14_5_D_arcref/outs/filtered_feature_bc_matrix")
SS_E15_A.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E15_A_arcref/outs/filtered_feature_bc_matrix")
SS_E15_B.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E15_B_arcref/outs/filtered_feature_bc_matrix")
SS_E15_C.data<-Read10X("/work/project/fragencode/workspace/ramiro_gastrulation/cellranger_count/SS_E15_C_arcref/outs/filtered_feature_bc_matrix")

In [59]:
j11.1 <- CreateSeuratObject(counts = j11.1.data)
j5_05122021 <- CreateSeuratObject(counts = j5_05122021.data)
j7_31082017 <- CreateSeuratObject(counts = j7_31082017.data)
j9_3 <- CreateSeuratObject(counts = j9_3.data)
SS_E11_5_A <- CreateSeuratObject(counts = SS_E11_5_A.data)
SS_E11_5_B <- CreateSeuratObject(counts = SS_E11_5_B.data)
SS_E12_A <- CreateSeuratObject(counts = SS_E12_A.data)
SS_E13_5_B <- CreateSeuratObject(counts = SS_E13_5_B.data)
SS_E13_A <- CreateSeuratObject(counts = SS_E13_A.data)
SS_E13_D <- CreateSeuratObject(counts = SS_E13_D.data)
SS_E14_5_B <- CreateSeuratObject(counts = SS_E14_5_B.data)
SS_E14_A <- CreateSeuratObject(counts = SS_E14_A.data)
SS_E15_B <- CreateSeuratObject(counts = SS_E15_B.data)
j11_2 <- CreateSeuratObject(counts = j11_2.data)
j7_15112017 <- CreateSeuratObject(counts = j7_15112017.data)
j7_4 <- CreateSeuratObject(counts = j7_4.data)
j9_3e <- CreateSeuratObject(counts = j9_3e.data)
SS_E12_5_A <- CreateSeuratObject(counts = SS_E12_5_A.data)
SS_E12_B <- CreateSeuratObject(counts = SS_E12_B.data)
SS_E13_5_C <- CreateSeuratObject(counts = SS_E13_5_C.data)
SS_E13_B <- CreateSeuratObject(counts = SS_E13_B.data)
SS_E13_E <- CreateSeuratObject(counts = SS_E13_E.data)
SS_E14_5_C <- CreateSeuratObject(counts = SS_E14_5_C.data)
SS_E14_B <- CreateSeuratObject(counts = SS_E14_B.data)
SS_E15_C <- CreateSeuratObject(counts = SS_E15_C.data)
j5_03062021 <- CreateSeuratObject(counts = j5_03062021.data)
j7_3 <- CreateSeuratObject(counts = j7_3.data)
j9_1 <- CreateSeuratObject(counts = j9_1.data)
j9_4 <- CreateSeuratObject(counts = j9_4.data)
SS_E12_5_B <- CreateSeuratObject(counts = SS_E12_5_B.data)
SS_E13_5_A <- CreateSeuratObject(counts = SS_E13_5_A.data)
SS_E13_5_D <- CreateSeuratObject(counts = SS_E13_5_D.data)
SS_E13_C <- CreateSeuratObject(counts = SS_E13_C.data)

SS_E14_5_D <- CreateSeuratObject(counts = SS_E14_5_D.data)
SS_E15_A <- CreateSeuratObject(counts = SS_E15_A.data)

Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with dashes ('-')"
Warning message:
"Feature names cannot have underscores ('_'), replacing with da

Annotation of samples

In [60]:
j11.1$SAMP <- "E11.1"
j5_05122021$SAMP <- "E5.1"
j7_31082017$SAMP <- "E7.1"
j9_3$SAMP <- "E9.3"
SS_E11_5_A$SAMP <- "J11.5A"
SS_E11_5_B$SAMP <- "J11.5B"
SS_E12_A$SAMP <- "E12.A"
SS_E13_5_B$SAMP <- "E13.5B"
SS_E13_A$SAMP <- "E13.A"
SS_E13_D$SAMP <- "E13.D"
SS_E14_5_B$SAMP <- "E14.5B"
SS_E14_A$SAMP <- "E14.A"
SS_E15_B$SAMP <- "E15.B"
j11_2$SAMP <- "E11.2"
j7_15112017$SAMP <- "E7.2"
j7_4$SAMP <- "E7.4"
j9_3e$SAMP <- "E9.3"
SS_E12_5_A$SAMP <- "E12.5.A"
SS_E12_B$SAMP <- "E12.B"
SS_E13_5_C$SAMP <- "E13.5.C"
SS_E13_B$SAMP <- "E13.B"
SS_E13_E$SAMP <- "E13.E"
SS_E14_5_C$SAMP <- "E14.5.C"
SS_E14_B$SAMP <- "E14.B"
SS_E15_C$SAMP <- "E14.C"
j5_03062021$SAMP <- "E5.2"
j7_3$SAMP <- "E7.3"
j9_1$SAMP <- "E9.1"
j9_4$SAMP <- "E9.4"
SS_E12_5_B$SAMP <- "E12.5.B"
SS_E13_5_A$SAMP <- "E13.5.A"
SS_E13_5_D$SAMP <- "E13.5.D"
SS_E13_C$SAMP <- "E13.C"

SS_E14_5_D$SAMP <- "E14.5.D"
SS_E15_A$SAMP <- "E15.A"

In [61]:
j11.1$new_TIME<-"E11"
j5_05122021$new_TIME<-"E5"
j7_31082017$new_TIME<-"E7"
j9_3$new_TIME<-"E9"
SS_E11_5_A$new_TIME<-"E11.5"
SS_E11_5_B$new_TIME<-"E11.5"
SS_E12_A$new_TIME<-"E12"
SS_E13_5_B$new_TIME<-"E13.5"
SS_E13_A$new_TIME<-"E13"
SS_E13_D$new_TIME<-"E13"
SS_E14_5_B$new_TIME<-"E14.5"
SS_E14_A$new_TIME<-"E14"
SS_E15_B$new_TIME<-"E15"
j11_2$new_TIME<-"E11"
j7_15112017$new_TIME<-"E7"
j7_4$new_TIME<-"E7"
j9_3e$new_TIME<-"E9"
SS_E12_5_A$new_TIME<-"E12.5"
SS_E12_B$new_TIME<-"E12"
SS_E13_5_C$new_TIME<-"E13.5"
SS_E13_B$new_TIME<-"E13"
SS_E13_E$new_TIME<-"E13"
SS_E14_5_C$new_TIME<-"E14.5"
SS_E14_B$new_TIME<-"E14"
SS_E15_C$new_TIME<-"E15"
j5_03062021$new_TIME<-"E5"
j7_3$new_TIME<-"E7"
j9_1$new_TIME<-"E9"
j9_4$new_TIME<-"E9"
SS_E12_5_B$new_TIME<-"E12.5"
SS_E13_5_A$new_TIME<-"E13.5"
SS_E13_5_D$new_TIME<-"E13.5"
SS_E13_C$new_TIME<-"E13"

SS_E14_5_D$new_TIME<-"E14.5"
SS_E15_A$new_TIME<-"E15"

# QC : mitochondrial genes

In [62]:
# Connect to AnnotationHub
ah <- AnnotationHub()

# Access the Ensembl database for organism
ahDb <- query(ah, 
              pattern = c("Sus scrofa", "EnsDb"), 
              ignore.case = TRUE)

In [63]:
id <- ahDb %>%
  mcols() %>%
  rownames() %>%
  tail(n = 1)

In [64]:
edb <- ah[[id]]

loading from cache



Extract gene-level information from database

In [65]:
annotations <- genes(edb, 
                     return.type = "data.frame") 

Select annotations of interest

In [66]:
annotations <- annotations %>%
  dplyr::select(gene_id, gene_name, gene_biotype, seq_name, description, entrezid)

Extract IDs for mitochondrial genes

In [67]:
mt <- annotations %>%
        dplyr::filter(seq_name == "MT")
mt$gene_name <- ifelse(mt$gene_name == '', mt$gene_id, mt$gene_name)
mt <- mt$gene_name

Count Mt RNA percentage

In [68]:
counts <- GetAssayData(object = j11.1, slot = "counts")
j11.1[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = j5_05122021, slot = "counts")
j5_05122021[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = j7_31082017, slot = "counts")
j7_31082017[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = j9_3, slot = "counts")
j9_3[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E11_5_A, slot = "counts")
SS_E11_5_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E11_5_B, slot = "counts")
SS_E11_5_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E12_A, slot = "counts")
SS_E12_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_5_B, slot = "counts")
SS_E13_5_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_A, slot = "counts")
SS_E13_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_D, slot = "counts")
SS_E13_D[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E14_5_B, slot = "counts")
SS_E14_5_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E14_A, slot = "counts")
SS_E14_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E15_B, slot = "counts")
SS_E15_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = j11_2, slot = "counts")
j11_2[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = j7_15112017, slot = "counts")
j7_15112017[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = j7_4, slot = "counts")
j7_4[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = j9_3e, slot = "counts")
j9_3e[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E12_5_A, slot = "counts")
SS_E12_5_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E12_B, slot = "counts")
SS_E12_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_5_C, slot = "counts")
SS_E13_5_C[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_B, slot = "counts")
SS_E13_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_E, slot = "counts")
SS_E13_E[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E14_5_C, slot = "counts")
SS_E14_5_C[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E14_B, slot = "counts")
SS_E14_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E15_C, slot = "counts")
SS_E15_C[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = j5_03062021, slot = "counts")
j5_03062021[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = j7_3, slot = "counts")
j7_3[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = j9_1, slot = "counts")
j9_1[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = j9_4, slot = "counts")
j9_4[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E12_5_B, slot = "counts")
SS_E12_5_B[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_5_A, slot = "counts")
SS_E13_5_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_5_D, slot = "counts")
SS_E13_5_D[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E13_C, slot = "counts")
SS_E13_C[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)

counts <- GetAssayData(object = SS_E14_5_D, slot = "counts")
SS_E14_5_D[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)
counts <- GetAssayData(object = SS_E15_A, slot = "counts")
SS_E15_A[["percent.mt"]] <- Matrix::colSums(counts[which(rownames(counts) %in% mt),], na.rm = T)/Matrix::colSums(counts, na.rm = T)

# merging of the different samples

In [69]:
embryo_pregastrulation <- merge(x = j5_05122021, y = c(j5_03062021, j7_15112017, j7_31082017, j7_3, j7_4, j9_1, j9_3e, j9_3, j9_4, j11.1, j11_2))

Warning message in CheckDuplicateCellNames(object.list = objects):
"Some cell names are duplicated across objects provided. Renaming to enforce unique cell names."


In [70]:
embryo_gastrulation <- merge(x = SS_E11_5_A, y = c(SS_E11_5_B, SS_E12_A, SS_E12_B, SS_E12_5_A, SS_E12_5_B, SS_E13_A, SS_E13_B, SS_E13_C, SS_E13_D, SS_E13_E, SS_E13_5_A, SS_E13_5_B, SS_E13_5_C, SS_E13_5_D, SS_E14_A, SS_E14_B, SS_E14_5_B, SS_E14_5_C, SS_E14_5_D, SS_E15_A, SS_E15_B, SS_E15_C))
#SS_E14_5_A

Warning message in CheckDuplicateCellNames(object.list = objects):
"Some cell names are duplicated across objects provided. Renaming to enforce unique cell names."


In [71]:
gastrulation_metadata = read.csv('/home/adufour/work/gastrulation_data/pig_meta_data_new.csv',sep=',')

In [72]:
embryo_metadata = read.csv("/home/adufour/work/table/all_embryos_subclusters.csv",sep=',')

In [73]:
embryo_gastrulation <- subset(embryo_gastrulation, cells = gastrulation_metadata$cell_barcode)

In [74]:
embryo_pregastrulation <- subset(embryo_pregastrulation, cells = embryo_metadata$cell)

In [75]:
rownames(embryo_metadata) <- embryo_metadata$cell

In [76]:
rownames(gastrulation_metadata) <- gastrulation_metadata$cell_barcode

In [77]:
embryo_pregastrulation@meta.data <- embryo_metadata[rownames(embryo_pregastrulation@meta.data),]
embryo_gastrulation@meta.data <- gastrulation_metadata[rownames(embryo_gastrulation@meta.data),]

In [78]:
embryo <- merge(embryo_pregastrulation, y = c(embryo_gastrulation), add.cell.ids = c("x", "y"))

In [31]:
prev <- readRDS("/home/adufour/work/rds_storage/gastrulation/embryo_E5_E15.rds")

In [79]:
embryo
prev

An object of class Seurat 
35670 features across 119157 samples within 1 assay 
Active assay: RNA (35670 features, 0 variable features)

An object of class Seurat 
21304 features across 126070 samples within 1 assay 
Active assay: RNA (21304 features, 0 variable features)

In [80]:
table(embryo@meta.data$Sample)
table(prev@meta.data$Sample)


      E11-1       E11-2  E11.5A.EPS  E11.5B.EPS  E12.5A.PS2  E12.5B.PS2 
       8938        6607        4801        4469         957         816 
    E12A.PS     E12B.PS   E13.5A.NG   E13.5B.NG   E13.5C.NG   E13.5D.NG 
        732        2399        1711        1815        6651        5070 
    E13A.NP     E13B.NP     E13C.NP     E13D.NP     E13E.NP E14.5B.8SOM 
        588        5050        4389       17296       16176        1236 
E14.5C.8SOM E14.5D.8SOM   E14A.6SOM   E14B.6SOM  E15A.10SOM  E15B.10SOM 
        898         762        1375        1112        1730        4502 
 E15C.10SOM        E5-1        E5-2        E7-1        E7-2        E7-3 
       3082         808         286         825         962         550 
       E7-4        E9-1        E9-2        E9-3        E9-4 
       1056         477        1762        2456        6813 


      E11-1       E11-2  E11.5A.EPS  E11.5B.EPS  E12.5A.PS2  E12.5B.PS2 
       9474        7233        4807        4487         957         816 
    E12A.PS     E12B.PS   E13.5A.NG   E13.5B.NG   E13.5C.NG   E13.5D.NG 
        776        2407        1711        1814        6687        8568 
    E13A.NP     E13B.NP     E13C.NP     E13D.NP     E13E.NP E14.5B.8SOM 
        588        5049        4389       17296       16176        1236 
E14.5C.8SOM E14.5D.8SOM   E14A.6SOM   E14B.6SOM  E15A.10SOM  E15B.10SOM 
        897         762        1375        1077        1730        4498 
 E15C.10SOM        E5-1        E5-2        E7-1        E7-2        E7-3 
       3079         919         307        1171         962         767 
       E7-4        E9-1        E9-2        E9-3        E9-4 
       1328         581        1969        2827        7350 

In [81]:
saveRDS(embryo, file = "/home/adufour/work/rds_storage/gastrulation/embryo_E5_E15_arc108.rds")